In [1]:
import awkward as ak
from coffea.nanoevents import NanoEventsFactory, NanoAODSchema

#fname = "root://eospublic.cern.ch//eos/opendata/cms/Run2016H/DoubleMuon/NANOAOD/UL2016_MiniAODv2_NanoAODv9-v1/2510000/127C2975-1B1C-A046-AABF-62B77E757A86.root"
fname = "file:/opendata_cms/DoubleMuon/Run2016H-UL2016_MiniAODv2_NanoAODv9-v1/NANOAOD/127C2975-1B1C-A046-AABF-62B77E757A86.root"
events = NanoEventsFactory.from_root(
    {fname: "Events"},
    schemaclass=NanoAODSchema,
    metadata={"dataset": "DoubleMuon"},
).events()

print("Number of events = ", ak.num(events.Muon, axis=0).compute())

/usr/local/venv/lib/python3.10/site-packages/coffea/nanoevents/schemas/fcc.py:5: FutureWarning: In version 2025.1.0 (target date: 2024-12-31 11:59:59-06:00), this will be an error.
To raise these warnings as errors (and get stack traces to find out where they're called), run
    import warnings
    warnings.filterwarnings("error", module="coffea.*")
after the first `import coffea` or use `@pytest.mark.filterwarnings("error:::coffea.*")` in pytest.
Issue: coffea.nanoevents.methods.vector will be removed and replaced with scikit-hep vector. Nanoevents schemas internal to coffea will be migrated. Otherwise please consider using that package!.
  from coffea.nanoevents.methods import vector
/usr/local/venv/lib/python3.10/site-packages/coffea/nanoevents/schemas/nanoaod.py:264: RuntimeWarning: Missing cross-reference index for LowPtElectron_electronIdx => Electron
  warnings.warn(
/usr/local/venv/lib/python3.10/site-packages/coffea/nanoevents/schemas/nanoaod.py:264: RuntimeWarning: Missing cr

Number of events =  2147195


In [2]:
print("Number of muons = ", ak.count(events.Muon.mass, axis=None).compute())

Number of muons =  4806013


In [3]:
selected_muons = events.Muon[(events.Muon.isGlobal == 1)]
print("Number of selected muons after isGlobal==1: ", ak.count(selected_muons.mass, axis=None).compute())

Number of selected muons after isGlobal==1:  4110800


In [4]:
selected_muons = selected_muons[(selected_muons.nStations > 0)]
print("Number of selected muons after nStations>0: ", ak.count(selected_muons.mass, axis=None).compute())

Number of selected muons after nStations>0:  3936437


In [5]:
selected_muons = selected_muons[(selected_muons.nTrackerLayers > 5)]
print("Number of selected muons after nTrackerLayers>5: ", ak.count(selected_muons.mass, axis=None).compute())

Number of selected muons after nTrackerLayers>5:  3908889


In [6]:
selected_muons = selected_muons[(selected_muons.dxy < 0.3) & (selected_muons.dz < 20.0)]
print("Number of selected muons after PV impact param sel: ", ak.count(selected_muons.mass, axis=None).compute())       

Number of selected muons after PV impact param sel:  3892547


In [7]:
selected_muons = selected_muons[(selected_muons.pt > 3.0) & (abs(selected_muons.eta) < 2.4)]
print("Number of selected muons after pt,eta sel: ", ak.count(selected_muons.mass, axis=None).compute())

Number of selected muons after pt,eta sel:  3883500


In [8]:
selected_muons = selected_muons[(selected_muons.pfRelIso03_chg < 0.5)]
print("Number of selected muons after ptcone30 isolation: ", ak.count(selected_muons.mass, axis=None).compute())

Number of selected muons after ptcone30 isolation:  2681124


In [9]:
selectede_muons = selected_muons[ak.num(selected_muons) > 0]
print("Number of selected events with num(muons)>0: ", ak.num(selectede_muons.mass, axis=0).compute())
print("Corresponding number of selected muons: ", ak.count(selectede_muons.mass, axis=None).compute())

Number of selected events with num(muons)>0:  1604214
Corresponding number of selected muons:  2681124


In [10]:
#selected_muons_c = selected_muons.compute()
pass

In [11]:
fourmuon = ak.combinations(selected_muons, 4, fields=['lep1','lep2','lep3','lep4'])

In [12]:
print(type(fourmuon))

<class 'dask_awkward.lib.core.Array'>


In [13]:
print("Number of 4-muon combination: ", ak.count(fourmuon.lep1.mass, axis=None).compute())

Number of 4-muon combination:  6170


In [14]:
fourmuon_c = fourmuon.compute()

In [15]:
print("Number of 4-muon (computed) combination: ", ak.count(fourmuon_c.lep1.mass, axis=None))

Number of 4-muon (computed) combination:  6170


In [16]:
# should be the same as above
print("Number of 4-muon (computed) combination: ", ak.count(fourmuon_c.lep4.pt, axis=None))

Number of 4-muon (computed) combination:  6170


In [17]:
print(fourmuon_c.lep1.charge[ak.num(fourmuon_c)>1] + fourmuon_c.lep2.charge[ak.num(fourmuon_c)>1])

[[0, 0, 0, 0, -2], [-2, -2, -2, -2, -2], ..., [0, 0, ..., 0], [0, 0, 0, 0, -2]]


In [18]:
#fourmuon_c = fourmuon_c[((fourmuon_c.lep1.charge + fourmuon_c.lep2.charge == 0) and
#                         (fourmuon_c.lep3.charge + fourmuon_c.lep4.charge == 0))]
#fourmuon_c = fourmuon_c[(fourmuon_c.lep1.charge + fourmuon_c.lep2.charge == 0)]
fourmuon_c = fourmuon_c[(fourmuon_c.lep1.charge + fourmuon_c.lep2.charge == 0) &
                        (fourmuon_c.lep3.charge + fourmuon_c.lep4.charge == 0)]

In [19]:
xxx = fourmuon_c.lep1.charge + fourmuon_c.lep2.charge == 0

In [20]:
i = 0
for j, qquad in enumerate(fourmuon_c):
    if len(qquad) == 0:
        continue
    print(j, ':', '[', end=' ')
    for quad in qquad:
        print(quad.lep1.charge+quad.lep2.charge==0, quad.lep3.charge+quad.lep4.charge==0, end=' ')
    print(']')
    i = i+1
    if i>100:
        break

1570 : [ True True ]
2436 : [ True True ]
2938 : [ True True ]
3467 : [ True True ]
4601 : [ True True ]
7634 : [ True True True True True True ]
8682 : [ True True ]
13819 : [ True True True True ]
14605 : [ True True ]
19403 : [ True True ]
22850 : [ True True ]
25327 : [ True True ]
27318 : [ True True ]
31044 : [ True True ]
31450 : [ True True ]
32974 : [ True True ]
34121 : [ True True ]
34153 : [ True True ]
34888 : [ True True ]
38340 : [ True True ]
50116 : [ True True ]
50385 : [ True True ]
50945 : [ True True ]
52093 : [ True True ]
53013 : [ True True ]
55074 : [ True True ]
55500 : [ True True True True True True ]
55809 : [ True True ]
56043 : [ True True ]
56242 : [ True True ]
56558 : [ True True ]
56691 : [ True True ]
56920 : [ True True ]
57181 : [ True True ]
57259 : [ True True ]
60537 : [ True True ]
61028 : [ True True ]
61077 : [ True True ]
62084 : [ True True ]
62366 : [ True True ]
62452 : [ True True ]
63684 : [ True True ]
63915 : [ True True ]
65921 : [ T

In [21]:
countset = set()
for j, qquad in enumerate(fourmuon_c):
    countset.add(len(qquad))
print(countset)

{0, 1, 2, 3, 6, 8, 12}


In [22]:
counthist = {}
for qquad in fourmuon_c:
    j = len(qquad)
    if j in counthist:
        counthist[j] += 1
    else:
        counthist[j] = 1
print(counthist)

{0: 2145060, 1: 2033, 3: 47, 2: 52, 12: 1, 8: 1, 6: 1}


In [23]:
i = 0
for j, qquad in enumerate(fourmuon_c):
    if len(qquad) == 0:
        continue
    print(j, ':', '[', end=' ')
    for quad in qquad:
        print(type(quad), end=' ')
        print(type(quad.lep1), end=' ')
        print(type(quad.lep1.pt), end=' ')
        print(quad.lep1.pt, quad.lep2.pt, quad.lep3.pt, quad.lep4.pt, end=' ')
    print(']')
    i = i+1
    if i>100:
        break

1570 : [ <class 'awkward.highlevel.Record'> <class 'coffea.nanoevents.methods.nanoaod.MuonRecord'> 

AttributeError: no field named 'MomentumClass'

In [24]:
print(fourmuon_c[1570].lep1.pt)

[4.44]


In [25]:
print(type(fourmuon_c[1570].lep1.pt))

<class 'awkward.highlevel.Array'>


In [26]:
i = 0
for j, qquad in enumerate(fourmuon_c):
    if len(qquad) == 0:
        continue
    print(j, ':', qquad.lep1.pt, qquad.lep2.pt, qquad.lep3.pt, qquad.lep4.pt)
    i += 1
    if i>100:
        break

1570 : [4.44] [4.41] [4.11] [3.79]
2436 : [15.4] [13.3] [7.14] [3.46]
2938 : [7.43] [5.83] [3.74] [3.41]
3467 : [4.01] [3.75] [3.33] [3.11]
4601 : [22.8] [13] [5.89] [3.47]
7634 : [15.7, 15.7, 15.7] [14.1, 14.1, 9.49] [9.49, 8.86, 8.86] [8.86, 5.8, 5.8]
8682 : [20.1] [9.27] [6.66] [6.03]
13819 : [21.4, 21.4] [8.09, 8.09] [6.74, 6.74] [3.8, 3.04]
14605 : [4.43] [4.22] [4.19] [4.19]
19403 : [26.3] [11.9] [9.06] [3.88]
22850 : [5.22] [4.71] [3.21] [3.01]
25327 : [9] [6.25] [5.42] [3.36]
27318 : [18.4] [9.03] [4.53] [3.71]
31044 : [20.3] [6.2] [3.32] [3.27]
31450 : [11.8] [6.63] [5.16] [3.87]
32974 : [58.2] [24.3] [12] [4.13]
34121 : [17.2] [14.4] [13.1] [5.54]
34153 : [8.34] [7.76] [6.35] [3.73]
34888 : [11.6] [8.45] [5.13] [3.73]
38340 : [9.3] [4.92] [4.07] [3.09]
50116 : [11.5] [6.55] [6.55] [5.67]
50385 : [20.3] [18] [8.33] [3.55]
50945 : [8.38] [4.3] [4.28] [3.36]
52093 : [13] [11.3] [7.01] [3.69]
53013 : [5.77] [4.62] [4.35] [3.54]
55074 : [7.16] [6.6] [5.06] [4.92]
55500 : [13.1, 13

In [27]:
fourmuon_c["pair1"] = fourmuon_c.lep1 + fourmuon_c.lep2
fourmuon_c["pair2"] = fourmuon_c.lep3 + fourmuon_c.lep4

In [28]:
i = 0
for j, qquad in enumerate(fourmuon_c):
    if len(qquad) == 0:
        continue
    print(j, ':', qquad.pair1.mass, qquad.pair2.mass)
    i += 1
    if i>200:
        break

1570 : [3.86] [7.6]
2436 : [3.11] [2.12]
2938 : [2.39] [7.77]
3467 : [16.4] [23.4]
4601 : [2.98] [15.5]
7634 : [3.06, 3.06, 19.7] [2.7, 14.4, 14.4]
8682 : [28] [1.95]
13819 : [26.4, 26.4] [4.76, 3.48]
14605 : [9.23] [7.3]
19403 : [48] [2.16]
22850 : [5.31] [4.71]
25327 : [16.4] [16.3]
27318 : [2.2] [2.12]
31044 : [18.9] [6.48]
31450 : [2.81] [38]
32974 : [90.9] [1.45]
34121 : [30.6] [15]
34153 : [4.69] [13.5]
34888 : [3.73] [10.6]
38340 : [14.6] [15.7]
50116 : [17.3] [12.2]
50385 : [41.4] [7.72]
50945 : [7.4] [6.74]
52093 : [3.06] [22.8]
53013 : [7.18] [8.44]
55074 : [14.1] [9.64]
55500 : [42.3, 42.3, 12] [9.2, 1.62, 1.62]
55809 : [8.8] [28.2]
56043 : [3.11] [15.8]
56242 : [24.3] [9.51]
56558 : [50.6] [1.09]
56691 : [2.69] [17.6]
56920 : [76.6] [28.6]
57181 : [3.8] [13]
57259 : [90.1] [14.8]
60537 : [13.8] [14.6]
61028 : [23.1] [0.947]
61077 : [24.1] [4.17]
62084 : [44.6] [21.4]
62366 : [6.02] [8.43]
62452 : [38.6] [28]
63684 : [65.8] [7.65]
63915 : [33.4] [73.6]
65921 : [47.4] [7.12]


In [29]:
i = 0
for j, qquad in enumerate(fourmuon_c):
    if len(qquad) == 0:
        continue
    print(j, ':', qquad.pair1.mass[0], qquad.pair2.mass[0])
    i += 1
    if i>30:
        break

1570 : 3.8560288 7.5953736
2436 : 3.1098068 2.1201692
2938 : 2.393349 7.7710075
3467 : 16.43877 23.356333
4601 : 2.9752808 15.549328
7634 : 3.055996 2.6950748
8682 : 27.983902 1.9456105
13819 : 26.369946 4.758242
14605 : 9.2253685 7.296944
19403 : 48.02938 2.1630223
22850 : 5.3092074 4.713311
25327 : 16.447248 16.28498
27318 : 2.2045512 2.1221833
31044 : 18.920177 6.475196
31450 : 2.8126898 38.02492
32974 : 90.88312 1.446199
34121 : 30.638048 15.000087
34153 : 4.691203 13.477272
34888 : 3.7286992 10.564054
38340 : 14.622793 15.748372
50116 : 17.305595 12.185983
50385 : 41.398808 7.7201886
50945 : 7.401854 6.738733
52093 : 3.0567248 22.83253
53013 : 7.177885 8.435709
55074 : 14.056404 9.636097
55500 : 42.303074 9.196771
55809 : 8.796528 28.154205
56043 : 3.1144547 15.787054
56242 : 24.345057 9.511949
56558 : 50.6359 1.0926893
